$$\mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \right)$$

<center><img src="tmp/viz_objects/1.png" alt="Input" style="width: 500px;"/></center>

<center><img src="tmp/viz_objects/2.png" alt="Add-Self-Loop" style="width: 500px;"/></center>

<center><img src="tmp/viz_objects/3.png" alt="Normalize" style="width: 500px;"/></center>

<center><img src="tmp/viz_objects/3_simplify.png" alt="Simplify" style="width: 500px;"/></center>

<center><img src="tmp/viz_objects/4.png" alt="Convolve" style="width: 500px;"/></center>

$$\begin{pmatrix}
    1.15e+00 &  8.24e-03 \\
    1.69e+00 &  3.47e+00 \\
   -1.37e-01 &  5.43e+00 \\
    1.15e+00 &  8.24e-03 \\
    8.97e+00 & -5.86e+00 \\
    1.15e+00 &  8.24e-03 \\
    1.69e+00 &  3.47e+00 \\
   -1.37e-01 &  5.43e+00 \\
    8.97e+00 & -5.86e+00
    \end{pmatrix}
    \bullet
 \begin{pmatrix}
 0.4264\\
 1.0660\\
 0.4330\\
 0.2132\\
 1.4434\\
 0.0909\\
 0.1250\\
 0.1667\\
 0.5000\\
\end{pmatrix}
=
\begin{pmatrix}
 4.92e-01 &  3.51e-03\\
 1.81e+00 &  3.69e+00\\
-5.94e-02 &  2.35e+00\\
 2.46e-01 &  1.75e-03\\
 1.29e+01 & -8.46e+00\\
 1.05e-01 &  7.49e-04\\
 2.12e-01 &  4.33e-01\\
-2.28e-02 &  9.05e-01\\
 4.48e+00 & -2.93e+00\\
\end{pmatrix}$$